In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import r2_score
import pickle

In [8]:
def pred(list_):
    
    df = pd.read_csv('despesas.csv')
    variaveis = df.columns

    variaveis_num = []
    variaveis_cat = []
    for i in variaveis:
        if (df[i].dtype == 'int64') or (df[i].dtype == 'float64'):
            variaveis_num.append(i)
        else:
            variaveis_cat.append(i)

    outlier_positive = df.gastos.mean() + (2*df.gastos.std())
    outlier_negative = df.gastos.mean() - (2*df.gastos.std())

    df_fumante = df[df.fumante == 'sim']
    df_fumante.gastos = np.where(df_fumante.gastos > outlier_positive, df_fumante.gastos.median(), df_fumante.gastos)
    df_nao_fumante = df[df.fumante == 'nao']
    df_nao_fumante.gastos = np.where(df_nao_fumante.gastos > outlier_positive, df_nao_fumante.gastos.median(), df_nao_fumante.gastos)
    df_new = pd.concat([df_fumante, df_nao_fumante])

    from sklearn.preprocessing import LabelEncoder
    from sklearn.pipeline import Pipeline

    class MultiColumnLabelEncoder:
        def __init__(self,columns = None):
            self.columns = columns # array of column names to encode

        def fit(self,X,y=None):
            return self # not relevant here

        def transform(self,X):
            '''
            Transforms columns of X specified in self.columns using
            LabelEncoder(). If no columns specified, transforms all
            columns in X.
            '''
            output = X.copy()
            if self.columns is not None:
                for col in self.columns:
                    output[col] = LabelEncoder().fit_transform(output[col])
            else:
                for colname,col in output.iteritems():
                    output[colname] = LabelEncoder().fit_transform(col)
            return output

        def fit_transform(self,X,y=None):

            return self.fit(X,y).transform(X)

    def bmi30(x):
        if x > 30:
            return 1
        else:
            return 0


    def transform_dataset(df):

        variaveis_num = []
        variaveis_cat = []
        for i in variaveis:
            if (df[i].dtype == 'int64') or (df[i].dtype == 'float64'):
                variaveis_num.append(i)
            else:
                variaveis_cat.append(i)

        df_ = MultiColumnLabelEncoder(columns = variaveis_cat).fit_transform(df)

        df_['idade2'] = df_['idade']*2
        df_['bmi30'] = df.bmi.apply(lambda x: bmi30(x))

        return df_

    loaded_model = pickle.load(open('qualirede_model.sav', 'rb'))

    df_prev = df.iloc[[0]]

    df_prev.loc[:,['idade']] = list_[0]
    df_prev.loc[:,['sexo']] = list_[1]
    df_prev.loc[:,['bmi']] = list_[2]
    df_prev.loc[:,['filhos']] = list_[3]
    df_prev.loc[:,['fumante']] = list_[4]
    df_prev.loc[:,['regiao']] = list_[5]
    df_prev.loc[:,['gastos']] = np.nan

    df_prev_ = df.append(df_prev, ignore_index=True)
    df_prev_2 = transform_dataset(df_prev_)

    X_pred = df_prev_2.loc[:,['idade', 'sexo', 'bmi', 'filhos', 'fumante', 'regiao', 'idade2', 'bmi30']].values
    scaler = MinMaxScaler(feature_range = (0, 1))
    X_scaled = scaler.fit_transform(X_pred)

    value_pred = list(loaded_model.predict(X_scaled))[1338]
    value_pred = int(value_pred)
    print('\nR$ {}\n'.format(value_pred))

In [11]:
idade = 20
sexo = 'mulher'
imc = 32.0
filhos = 1
fumante = 'nao'
regiao = 'norte'

list_ = [idade, sexo, imc, filhos, fumante, regiao]

In [12]:
pred(list_)


R$ 5404

